In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget,QScrollArea,QTableWidget,QTableWidgetItem,QLabel, QPushButton, QTextEdit,QMessageBox,QFileDialog,QPlainTextEdit
from tkinter import ttk
from tkinter import Tk
from PIL import ImageTk, Image
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QTextCursor, QTextCharFormat, QColor, QFont

df = pd.DataFrame()
df_varazdin = pd.DataFrame()
df_koprivnica = pd.DataFrame()

class TreniranjeTesitranjeApp(QMainWindow):
    def __init__(self, parent=None):
        super().__init__()

        self.setWindowTitle("Treniranje i testiranje")
        self.setGeometry(100, 100, 300, 300)
        
      

        button_styles = '''
            QPushButton {
                background-color: #ff6347;
                color: white;
                border: none;
                border-radius: 5px;
                padding: 10px;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #ff4d26; 
            }
        '''

        self.button1 = QPushButton("Excel s podacima za Čakovec", self)
        self.button1.setGeometry(60, 52, 270, 40)
        self.button1.setStyleSheet(button_styles)
        self.button1.clicked.connect(self.select_df)

        self.button2 = QPushButton("Excel s podacima za Varaždin", self)
        self.button2.setGeometry(60, 102, 270, 40)
        self.button2.setStyleSheet(button_styles)
        self.button2.clicked.connect(self.select_df_varazdin)

        self.button3 = QPushButton("Excel s podacima za Koprivnicu", self)
        self.button3.setGeometry(60, 152, 270, 40)
        self.button3.setStyleSheet(button_styles)
        self.button3.clicked.connect(self.select_df_koprivnica)

        self.button4 = QPushButton("Trening i testiranje", self)
        self.button4.setGeometry(60, 202, 270, 40)
        self.button4.setStyleSheet(button_styles)
        self.button4.clicked.connect(self.predict_weather)

    def select_df(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Odaberi excel čakovec datoteku", "", "CSV Files (*.csv)")

        if file_path:
            self.load_df(file_path)

    def load_df(self, file_path):
        global df
        df = pd.read_csv(file_path, encoding='cp1250', delimiter=";")

    def select_df_varazdin(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Odaberi excel varaždin datoteku", "", "CSV Files (*.csv)")

        if file_path:
            self.load_df_varazdin(file_path)

    def load_df_varazdin(self, file_path):
        global df_varazdin
        df_varazdin = pd.read_csv(file_path, encoding='cp1250', delimiter=";")

    def select_df_koprivnica(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Odaberi excel koprivnica datoteku", "", "CSV Files (*.csv)")
        
        if file_path:
            self.load_df_koprivnica(file_path)

    def load_df_koprivnica(self, file_path):
        global df_koprivnica
        df_koprivnica = pd.read_csv(file_path, encoding='cp1250', delimiter=";")

    def predict_weather(self):
        global df, df_varazdin, df_koprivnica, new_df_2024,r2,mse,new_df,merged_df

        if df.empty or df_varazdin.empty or df_koprivnica.empty:
            QMessageBox.warning(self, "Upozorenje", "Molimo odaberite sve excel datoteke.")
            return
        df['Mjesto']
        df['Dan'] 
        for index, row in df.iterrows():
            if row['Mjesto'] == 'Čakovec':
                df.at[index, 'Mjesto'] = 'Čakovec'
            elif row['Mjesto'] == 'Varaždin':
                df.at[index, 'Mjesto'] = 'Varaždin'
            elif row['Mjesto'] == 'Koprivnica':
                df.at[index, 'Mjesto'] = 'Koprivnica'


        df = pd.concat([df, df_varazdin, df_koprivnica], ignore_index=True)

        df['Datum'] = pd.date_range(start=datetime(2023, 4, 1), periods=len(df))
        df['Trenutnovrijeme'] = df['Trenutnovrijeme'].replace(['Kiša', 'Oblačno', 'Poluoblačno', 'Slabi pljuskovi uz kišu', 'Sunčano',
       'Uglavnom oblačno', 'Uglavnom sunčano', 'Vedro',
       'Vedro s povremenom naoblakom'],[0,1,2,3,4,5,6,7,8])

        X = ['TrenutačnaTemperatura','Oborine', 'Vlaga', 'Vjetar','Trenutnovrijeme','Najvećatemperatura', 'Najmanjatemperatura']
        y = ['TrenutačnaTemperatura','Oborine', 'Vlaga', 'Vjetar','Trenutnovrijeme','Najvećatemperatura', 'Najmanjatemperatura']


        X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.3, random_state=43)


        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)


        mlp = MLPRegressor(hidden_layer_sizes=(100,50,100, 50), max_iter=1500, solver='adam', activation='relu', random_state=43)
        mlp.fit(X_train, y_train)


        y_pred = mlp.predict(X_test)


        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print('Mean Squared Error:', mse)
        print('R^2:', r2)





        new_dates = pd.date_range(start=datetime(2024, 4, 1), periods=len(df))
        new_df = df.copy()


        new_X = scaler.transform(new_df[X])
        new_y = mlp.predict(new_X)

        for i, param in enumerate(y):
            new_df[param] = new_y[:, i]



        trenutnovrijeme_dict = {
            0: 'Kiša',
            1: 'Oblačno',
            2: 'Poluoblačno',
            3: 'Slabi pljuskovi uz kišu',
            4: 'Sunčano',
            5: 'Uglavnom oblačno',
            6: 'Uglavnom sunčano',
            7: 'Vedro',
            8: 'Vedro s povremenom naoblakom'
        }


        merged_df = pd.concat([df, new_df], ignore_index=True).round()
        merged_df=merged_df.round(2)

        merged_df['Trenutnovrijeme'] = merged_df['Trenutnovrijeme'].replace(trenutnovrijeme_dict)
        merged_df['Oborine'] = merged_df['Oborine'].apply(lambda x: abs(x))
        new_df_2024 = merged_df.iloc[len(df):].reset_index(drop=True)
        new_dates_2024 = pd.date_range(start=pd.Timestamp(2024, 4, 1), periods=len(new_df_2024), freq='D')
        new_df_2024['Datum'] = new_dates_2024
        new_df_2024 = new_df_2024[y].round(2)
        new_df_2024.insert(7, 'Datum', new_dates_2024)
        new_df_2024['Mjesto'] = df['Mjesto']
        new_df_2024['Dan'] = df['Dan']
   

        QMessageBox.information(self, "Treniranje i testiranje", "Treniranje i testiranje je završeno, predviđeni rezultati su spremni za prikazivanje.")
        pass
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Glavno sučelje")
        self.setGeometry(100, 100, 1600, 900)
        image = QImage('C:/Users/fbara/Desktop/images.png')
        image = image.scaled(1600, 900)
        label = QLabel(self)
        label.setPixmap(QPixmap.fromImage(image))
        label.setGeometry(0, 0, 1600, 900)
        label.setAlignment(Qt.AlignCenter)
        image_opacity = 0.7 
        button_styles = '''
            QPushButton {
                background-color: #ff6347; /* Promijenite boju ovdje */
                color: white;
                border: none;
                border-radius: 5px;
                padding: 10px;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #ff4d26; /* Promijenite boju ovdje */
            }
        '''
        self.button4 = QPushButton("Treniranje i Testiranje", self)
        self.button4.setGeometry(700, 300, 250, 40)
        self.button4.setStyleSheet(button_styles)
        self.button4.clicked.connect(self.show_interface5)
        self.button1 = QPushButton("Statistika", self)
        self.button1.setGeometry(700, 360, 250, 40)
        self.button1.setStyleSheet(button_styles)
        self.button1.clicked.connect(self.show_interface1)
        self.button2 = QPushButton("Vrijeme", self)
        self.button2.setGeometry(700, 420, 250, 40)
        self.button2.setStyleSheet(button_styles)
        self.button2.clicked.connect(self.show_interface2)
        self.button3 = QPushButton("Vremenska prognoza", self)
        self.button3.setGeometry(700, 480, 250, 40)
        self.button3.setStyleSheet(button_styles)
        self.button3.clicked.connect(self.show_interface3)
        self.button = QPushButton("Treniranje-testiranje-pojedinačno", self)
        self.button.setGeometry(700, 540, 250, 40)
        self.button.setStyleSheet(button_styles)
        self.button.clicked.connect(self.show_interface4)
        self.current_interface = None

    def show_interface1(self):
        if df.empty or df_varazdin.empty or df_koprivnica.empty:
            QMessageBox.warning(self, "Upozorenje", "Molimo odaberite excel datoteke kako bi ste vidjeli Statistiku.")
            return
        if self.current_interface:
            self.current_interface.hide()
            self.current_interface = None

        self.current_interface = StatistikaApp(self)
        self.current_interface.setGeometry(300, 50, 400, 400)
        self.current_interface.show()
        

    def show_interface2(self):
        if df.empty or df_varazdin.empty or df_koprivnica.empty:
            QMessageBox.warning(self, "Upozorenje", "Molimo odaberite excel datoteke kako bi ste vidjeli Vrijeme.")
            return
        if self.current_interface:
            self.current_interface.hide()
            self.current_interface = None

        self.current_interface = PrvoSučelje(df, new_df_2024, places, self)
        self.current_interface.setGeometry(300, 50, 400, 400)
        self.current_interface.show()
    
        pass

    def show_interface3(self):
        if df.empty or df_varazdin.empty or df_koprivnica.empty:
            QMessageBox.warning(self, "Upozorenje", "odaberite excel datoteke kako bi ste vidjeli Prognozu vremena.")
            return
        if self.current_interface:
            self.current_interface.hide()
            self.current_interface = None

        self.current_interface = third_interface()
        pass
    
    def show_interface4(self):
        if self.current_interface:
            self.current_interface.hide()
            self.current_interface = None

        self.current_interface = Test(self)
        self.current_interface.setGeometry(300, 50, 400, 400)
        self.current_interface.show()
        
    def show_interface5(self):
        if self.current_interface:
            self.current_interface.hide()
            self.current_interface = None

        self.current_interface = TreniranjeTesitranjeApp(self)
        self.current_interface.setGeometry(300, 50, 400, 400)
        self.current_interface.show()


        
class StatistikaApp(QMainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Statistika 2023")
        self.setGeometry(100, 100, 400, 400)
        main_widget = QWidget()
        main_layout = QVBoxLayout()
        main_widget.setLayout(main_layout)
        self.setCentralWidget(main_widget)
        
        button_styles = '''
            QPushButton {
                background-color: #ff6347; /* Promijenite boju ovdje */
                color: white;
                border: none;
                border-radius: 5px;
                padding: 10px;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #ff4d26; /* Promijenite boju ovdje */
            }
        '''
        
        self.r2_mse_button = QPushButton("Prikaži R^2 i MSE", self)
        self.r2_mse_button.setStyleSheet(button_styles)
        self.r2_mse_button.clicked.connect(self.r2_mse)

        self.statistika_2023_button = QPushButton("Prikaži statistiku za 2023", self)
        self.statistika_2023_button.setStyleSheet(button_styles)
        self.statistika_2023_button.clicked.connect(self.prikazi_statistiku_2023)

        self.statistika_2024_button = QPushButton("Prikaži statistiku za 2024", self)
        self.statistika_2024_button.setStyleSheet(button_styles)
        self.statistika_2024_button.clicked.connect(self.prikazi_statistiku_2024)

        self.result_label = QLabel(self)

        self.result_text = QTextEdit(self)
        self.result_text.setReadOnly(True)

        main_layout.addWidget(self.r2_mse_button)
        main_layout.addWidget(self.statistika_2023_button)
        main_layout.addWidget(self.statistika_2024_button)
        main_layout.addWidget(self.result_label)
        main_layout.addWidget(self.result_text)

    def r2_mse(self):
        self.result_label.setText(f"Koeficijent determinacije i Srednja kvadratna greška: {r2}, {mse}")
        pass

    def prikazi_statistiku_2023(self):
        parameters = [
            'TrenutačnaTemperatura', 'Oborine', 'Vlaga', 'Vjetar', 'Trenutnovrijeme',
            'Najvećatemperatura', 'Najmanjatemperatura'
        ]
        places = df['Mjesto'].unique()
        self.result_text.clear()
        for parameter in parameters:
            if parameter == 'Trenutnovrijeme':
                for place in places:
                    df_place = df.loc[df['Mjesto'] == place]
                    postotak_kisa = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 0 else 0).mean() * 100
                    postotak_oblacno = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 1 else 0).mean() * 100
                    postotak_poluoblacno = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 2 else 0).mean() * 100
                    postotak_slabipljuskovi = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 3 else 0).mean() * 100
                    postotak_suncano = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 4 else 0).mean() * 100
                    postotak_uglavnomoblacno = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 5 else 0).mean() * 100
                    postotak_uglavnomsuncano = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 6 else 0).mean() * 100
                    postotak_vedro = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 7 else 0).mean() * 100
                    postotak_vedronaoblaka = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 8 else 0).mean() * 100

                    self.result_text.append(f'Postotak kišnih dana u {place}: {postotak_kisa:.2f}%\n')
                    self.result_text.append(f'Postotak oblačnih dana u {place}: {postotak_oblacno:.2f}%\n')
                    self.result_text.append(f'Postotak poluoblačnih dana u {place}: {postotak_poluoblacno:.2f}%\n')
                    self.result_text.append(f'Postotak slabih pljuskova dana u {place}: {postotak_slabipljuskovi:.2f}%\n')
                    self.result_text.append(f'Postotak sunčanih dana u {place}: {postotak_suncano:.2f}%\n')
                    self.result_text.append(f'Postotak uglavnom oblačnih dana u {place}: {postotak_uglavnomoblacno:.2f}%\n')
                    self.result_text.append(f'Postotak uglavnomo sunčanih dana u {place}: {postotak_uglavnomsuncano:.2f}%\n')
                    self.result_text.append(f'Postotak vedrih dana u {place}: {postotak_vedro:.2f}%\n')
                    self.result_text.append(f'Postotak vedro s naoblakom dana u {place}: {postotak_vedronaoblaka:.2f}%\n')
                    self.result_text.append('\n')
            else:
                for place in places:
                    df_place = df.loc[df['Mjesto'] == place]
                    mean_value = df_place[parameter].mean()
                    self.result_text.append(f'Prosječna vrijednost parametra {parameter} u {place}: {mean_value:.2f}\n')

                self.result_text.append('\n')
        
        pass

    def prikazi_statistiku_2024(self):
        parameters = ['TrenutačnaTemperatura', 'Oborine', 'Vlaga', 'Vjetar', 'Trenutnovrijeme', 'Najvećatemperatura', 'Najmanjatemperatura']
        places = new_df_2024['Mjesto'].unique()

        self.result_text.clear()

        for parameter in parameters:
            if parameter == 'Trenutnovrijeme':
                for place in places:
                    df_place = new_df_2024.loc[df['Mjesto'] == place]
                    postotak_kisa = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Kiša' else 0).mean() * 100
                    postotak_oblacno = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Oblačno' else 0).mean() * 100
                    postotak_poluoblacno = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Poluoblačno' else 0).mean() * 100
                    postotak_slabipljuskovi = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Slabi pljuskovi uz kišu' else 0).mean() * 100
                    postotak_suncano = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Sunčano' else 0).mean() * 100
                    postotak_uglavnomoblacno = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Uglavnom oblačno' else 0).mean() * 100
                    postotak_uglavnomsuncano = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Uglavnom sunčano' else 0).mean() * 100
                    postotak_vedro = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Vedro' else 0).mean() * 100
                    postotak_vedronaoblaka = df_place['Trenutnovrijeme'].apply(lambda x: 1 if x == 'Vedro s povremenom naoblakom' else 0).mean() * 100

                    self.result_text.append('Postotak kišnih dana u {}: {:.2f}%\n'.format(place, postotak_kisa))
                    self.result_text.append('Postotak oblačnih dana u {}: {:.2f}%\n'.format(place, postotak_oblacno))
                    self.result_text.append('Postotak poluoblačnih dana u {}: {:.2f}%\n'.format(place, postotak_poluoblacno))
                    self.result_text.append('Postotak slabih pljuskova dana u {}: {:.2f}%\n'.format(place, postotak_slabipljuskovi))
                    self.result_text.append('Postotak sunčanih dana u {}: {:.2f}%\n'.format(place, postotak_suncano))
                    self.result_text.append('Postotak uglavnom oblačnih dana u {}: {:.2f}%\n'.format(place, postotak_uglavnomoblacno))
                    self.result_text.append('Postotak uglavnomo sunčanih dana u {}: {:.2f}%\n'.format(place, postotak_uglavnomsuncano))
                    self.result_text.append('Postotak vedrih dana u {}: {:.2f}%\n'.format(place, postotak_vedro))
                    self.result_text.append('Postotak vedro s naoblakom dana u {}: {:.2f}%\n'.format(place, postotak_vedronaoblaka))
                    self.result_text.append('\n')
            else:
                for place in places:
                    df_place = new_df_2024.loc[df['Mjesto'] == place]
                    mean_value = df_place[parameter].mean()
                    self.result_text.append('Prosječna vrijednost parametra {} u {}: {:.2f}\n'.format(parameter, place, mean_value))
                self.result_text.append('\n')
                
                
class PrvoSučelje(QMainWindow):
    def __init__(self,df,new_df_2024,places,parent=None):
        super().__init__(parent)
    
        self.places = places
        self.df = df
        self.new_df_2024 = new_df_2024
        self.places = places
        self.current_place_index = 0
        self.setWindowTitle("Vrijeme")
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout()
        self.central_widget.setLayout(self.layout)
        
        button_styles = '''
            QPushButton {
                background-color: #ff6347; /* Promijenite boju ovdje */
                color: white;
                border: none;
                border-radius: 5px;
                padding: 10px;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #ff4d26; /* Promijenite boju ovdje */
            }
        '''

        self.place_label = QLabel()
        self.layout.addWidget(self.place_label)

        self.table = QTableWidget()
        self.layout.addWidget(self.table)

        self.next_button = QPushButton("Sljedeće mjesto")
        self.next_button.setStyleSheet(button_styles)
        self.next_button.clicked.connect(self.show_next_place)
        self.layout.addWidget(self.next_button)
        self.show_weather()
   
        
        
        
    def show_weather(self):
        place = self.places[self.current_place_index]
        start_date_2023 = datetime(2023, 4, 1)
        end_date_2023 = datetime(2023, 4, 30)
        new_df_place_2023 = self.df.loc[self.df['Mjesto'] == place].assign(Datum=pd.date_range(start=start_date_2023, end=end_date_2023, freq='D'))

        start_date_2024 = datetime(2024, 4, 1)
        end_date_2024 = datetime(2024, 4, 30)
        new_df_place_2024 = self.new_df_2024.loc[self.new_df_2024['Mjesto'] == place].assign(Datum=pd.date_range(start=start_date_2024, end=end_date_2024, freq='D'))

        trenutnovrijeme_dict = {
            0: 'Kiša', 1: 'Oblačno', 2: 'Poluoblačno', 3: 'Slabi pljuskovi uz kišu',
            4: 'Sunčano', 5: 'Uglavnom oblačno', 6: 'Uglavnom sunčano',
            7: 'Vedro', 8: 'Vedro s povremenom naoblakom'
        }

        smajlici = {
            'Kiša': '🌧️', 'Oblačno': '☁️', 'Poluoblačno': '⛅',
            'Slabi pljuskovi uz kišu': '🌦️', 'Sunčano': '☀️', 'Uglavnom oblačno': '🌥️',
            'Uglavnom sunčano': '🌤️', 'Vedro': '🌞', 'Vedro s povremenom naoblakom': '🌤️'
        }

        self.place_label.setText(place)  

        header_labels = ['Datum', '2023', f'2024 - {place}']
        self.table.setColumnCount(len(header_labels))
        self.table.setHorizontalHeaderLabels(header_labels)

        self.table.setRowCount(len(new_df_place_2023))
        self.table.verticalHeader().setVisible(False)

        for row, (_, row_2023), (_, row_2024) in zip(range(len(new_df_place_2023)), new_df_place_2023.iterrows(), new_df_place_2024.iterrows()):
            godina_str = row_2023['Datum'].strftime('%d.%m')
            godina_str_2023 = row_2023['Datum'].strftime('%Y')
            godina_str_2024 = row_2024['Datum'].strftime('%Y')

            trenutnovrijeme_2023 = trenutnovrijeme_dict.get(row_2023['Trenutnovrijeme'], '')
            smajlic_2023 = smajlici.get(trenutnovrijeme_2023, '')

            trenutnovrijeme_2024 = row_2024['Trenutnovrijeme']
            smajlic_2024 = smajlici.get(trenutnovrijeme_2024, '')

            datum_2023 = QTableWidgetItem(godina_str)
            promjena_2023 = QTableWidgetItem(f"{smajlic_2023} {trenutnovrijeme_2023}")
            promjena_2024 = QTableWidgetItem(f"{smajlic_2024} {trenutnovrijeme_2024}")

            if trenutnovrijeme_2023 != trenutnovrijeme_2024:
                font = datum_2023.font()
                font.setBold(True)
                datum_2023.setFont(font)
                promjena_2023.setFont(font)
                promjena_2024.setFont(font)

            self.table.setItem(row, 0, datum_2023)
            self.table.setItem(row, 1, promjena_2023)
            self.table.setItem(row, 2, promjena_2024)

        self.table.resizeColumnsToContents()
        self.table.resizeRowsToContents()

    def show_next_place(self):
        self.current_place_index = (self.current_place_index + 1) % len(self.places)
        self.show_weather()
        
places = ['Čakovec', 'Varaždin', 'Koprivnica']

def third_interface():
    root = tk.Tk()
    root.title("Vremenska prognoza")


    main_frame = tk.Frame(root)
    main_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)


    scrollbar = ttk.Scrollbar(main_frame, orient=tk.VERTICAL)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)


    canvas = tk.Canvas(main_frame, yscrollcommand=scrollbar.set)
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    canvas.configure(yscrollcommand=scrollbar.set)


    inner_frame = tk.Frame(canvas)
    canvas.create_window((0, 0), window=inner_frame, anchor='nw')

    scrollbar.configure(command=canvas.yview)
    canvas.configure(yscrollcommand=scrollbar.set)


    image = Image.open('C:/Users/fbara/Desktop/vremenska-prognoza-1.jpg')
    image = image.resize((1600, 800))
    photo = ImageTk.PhotoImage(image)
    label = tk.Label(canvas, image=photo)
    label.pack(fill="both", expand=True)

   

    
        
        
        


    def on_close(self):
        self.destroy()
        self.parent.deiconify()
        
    def stvar_pred_graph(inner_frame, df, new_df, place, parameter, ylabel):
                units = {
                    
                    'Oborine':'%',
                    'Vlaga': '%',
                    'Vjetar': 'km/h',
                    'Najvećatemperatura': '°C',
                    'Najmanjatemperatura': '°C'
    }
                fig = plt.figure(figsize=(10, 5))
                df_place = df.loc[df['Mjesto'] == place]
                new_df_place = new_df.loc[new_df['Mjesto'] == place]
                plt.plot(new_df_place['Datum'], df_place[parameter], label='Stvarne vrijednosti')
                plt.plot(new_df_place['Datum'], new_df_place[parameter], label='Predviđene vrijednosti')
                plt.xlabel('Datum')
                plt.ylabel(f'{parameter} ({units[parameter]})')
                plt.title('Stvarne i predviđene vrijednosti za {} - {}'.format(place, parameter))
                plt.legend()
                plt.close()
                canvas = FigureCanvasTkAgg(fig, master=inner_frame)
                canvas.draw()
                canvas.get_tk_widget().pack()



    parameters = ['Oborine', 'Vlaga', 'Vjetar', 'Najvećatemperatura', 'Najmanjatemperatura']
    places = ['Čakovec', 'Varaždin', 'Koprivnica']

    def prikazi_grafove(place):
        label.pack_forget()
        for parameter in parameters:
            inner_frame_graph = tk.Frame(inner_frame)
            inner_frame_graph.pack()

            if place == 'Varaždin':
                start_date = datetime(2024, 4, 1)
                end_date = datetime(2024, 4, 30)
                new_df_varazdin = new_df.loc[new_df['Mjesto'] == 'Varaždin']
                new_df_varazdin = new_df_varazdin.assign(Datum=pd.date_range(start=start_date, end=end_date, freq='D'))
                stvar_pred_graph(inner_frame_graph, df, new_df_varazdin, place, parameter, ylabel=parameter)
            elif place == 'Koprivnica':
                start_date = datetime(2024, 4, 1)
                end_date = datetime(2024, 4, 30)
                new_df_koprivnica = new_df.loc[new_df['Mjesto'] == 'Koprivnica']
                new_df_koprivnica = new_df_koprivnica.assign(Datum=pd.date_range(start=start_date, end=end_date, freq='D'))
                stvar_pred_graph(inner_frame_graph, df, new_df_koprivnica, place, parameter, ylabel=parameter)
            else:
          
                stvar_pred_graph(inner_frame_graph, df, new_df_2024, place, parameter, ylabel=parameter)


    def otvori_graf(mjesto):
            label.pack_forget()

            for widget in inner_frame.winfo_children():
                widget.destroy()

            prikazi_grafove(mjesto)


   
            for widget in inner_frame.winfo_children():
                if isinstance(widget, tk.Canvas):
                    widget.destroy()
    def prikazi_trenutno_vrijeme(df, new_df_2024, place):
            trenutnovrijeme_dict = {
                0: 'Kiša', 1: 'Oblačno', 2: 'Poluoblačno', 3: 'Slabi pljuskovi uz kišu',
                4: 'Sunčano', 5: 'Uglavnom oblačno', 6: 'Uglavnom sunčano',
                7: 'Vedro', 8: 'Vedro s povremenom naoblakom'
            }

            trenutno_vrijeme_2023 = df.loc[df['Mjesto'] == place, ['Datum', 'Trenutnovrijeme']]
            trenutno_vrijeme_2024 = new_df_2024.loc[new_df_2024['Mjesto'] == place, ['Datum', 'Trenutnovrijeme']]

            merged_df = trenutno_vrijeme_2023.merge(trenutno_vrijeme_2024, on='Datum', how='inner')

            fig = plt.figure(figsize=(12, 6))

            plt.plot(merged_df['Datum'], [trenutnovrijeme_dict.get(v, '') for v in merged_df['Trenutnovrijeme_x']], 'bo-', label='2023')
            plt.plot(merged_df['Datum'], merged_df['Trenutnovrijeme_y'], 'ro-', label='2024')

            plt.xlabel('Datum')
            plt.ylabel('Trenutno vrijeme')
            plt.title(f"Trenutno vrijeme u {place} - Usporedba 2023. i 2024.")
            plt.legend()

            plt.xticks(rotation=45)

            plt.tight_layout()
            plt.close()
            canvas = FigureCanvasTkAgg(fig, master=inner_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)




    def otvori_graff(mjesto):
            label.pack_forget()
            for widget in inner_frame.winfo_children():
                widget.destroy()

            start_date = datetime(2023, 4, 1)
            end_date = datetime(2023, 4, 30)
            new_df_place_2023 = df.loc[df['Mjesto'] == mjesto].assign(Datum=pd.date_range(start=start_date, end=end_date, freq='D'))

            start_date_2024 = datetime(2024, 4, 1)
            end_date_2024 = datetime(2024, 4, 30)
            new_df_place_2024 = new_df_2024.loc[new_df_2024['Mjesto'] == mjesto].assign(Datum=pd.date_range(start=start_date, end=end_date, freq='D'))

            prikazi_trenutno_vrijeme(new_df_place_2023, new_df_place_2024, mjesto)


    def prikazi_totalne_oborine_graf(inner_frame, total_rain_2023, total_rain_2024):
            label.pack_forget()
            bar_width = 0.35
            r1 = np.arange(len(total_rain_2023))
            r2 = [x + bar_width for x in r1]

            fig = plt.figure(figsize=(10, 5))

            plt.bar(r1, total_rain_2023.values, color='b', width=bar_width, edgecolor='white', label='2023')
            plt.bar(r2, total_rain_2024.values, color='r', width=bar_width, edgecolor='white', label='2024')
            plt.xlabel('Grad')
            plt.ylabel(f'Oborine u mm')
            plt.title('Ukupne količine oborina po gradu')
            plt.xticks([r + bar_width / 2 for r in range(len(total_rain_2023))], total_rain_2023.index)
            for i, v in enumerate(total_rain_2023.values):
                plt.text(r1[i] - 0.08, v + 1, str(v), color='blue', fontweight='bold')

            for i, v in enumerate(total_rain_2024.values):
                plt.text(r2[i] - 0.08, v + 1, str(v), color='red', fontweight='bold')

            plt.close()
            canvas = FigureCanvasTkAgg(fig, master=inner_frame)
            canvas.draw()
            canvas.get_tk_widget().pack()
    
    def prikazi_treci_graf():
            label.pack_forget()
            for widget in inner_frame.winfo_children():
                widget.destroy()
            total_rain_2023 = df.groupby('Mjesto')['Oborine'].sum()
            total_rain_2024 = new_df_2024.groupby('Mjesto')['Oborine'].sum()
            prikazi_totalne_oborine_graf(inner_frame, total_rain_2023, total_rain_2024)  
    
  
   
        
    graf_cakovec_button = tk.Button(root, text="Čakovec-Stvarno-Predviđeno", command=lambda: otvori_graf('Čakovec'))
    graf_cakovec_button.pack(side=tk.LEFT)

    graf_varazdin_button = tk.Button(root, text="Varaždin-Stvarno-Predviđeno", command=lambda: otvori_graf('Varaždin'))
    graf_varazdin_button.pack(side=tk.LEFT)

    graf_koprivnica_button = tk.Button(root, text="Koprivnica-Stvarno-Predviđeno", command=lambda: otvori_graf('Koprivnica'))
    graf_koprivnica_button.pack(side=tk.LEFT)
    cakovec_button = tk.Button(root, text="Čakovec-Vrijeme-Predviđeno", command=lambda: otvori_graff('Čakovec'))
    cakovec_button.pack(side=tk.LEFT)

    varazdin_button = tk.Button(root, text="Varaždin-Vrijeme-Predviđeno", command=lambda: otvori_graff('Varaždin'))
    varazdin_button.pack(side=tk.LEFT)

    koprivnica_button = tk.Button(root, text="Koprivnica-Vrijeme-Predviđeno", command=lambda: otvori_graff('Koprivnica'))
    koprivnica_button.pack(side=tk.LEFT)
    bar_button = tk.Button(root, text="Bar-Graf-Oborine-Predviđene", command=prikazi_treci_graf)
    bar_button.pack(side=tk.LEFT)
  
    
    canvas.configure(scrollregion=canvas.bbox("all"), yscrollcommand=scrollbar.set)


    inner_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))


    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
     
    root.mainloop()

class Test(QMainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("Treniranje-Testiranje")
        self.setGeometry(100, 100, 1200, 400)

        self.text_edit = QTextEdit(self)
        self.text_edit.setGeometry(10, 50, 580, 340)
        button_styles = '''
            QPushButton {
                background-color: #ff6347; /* Promijenite boju ovdje */
                color: white;
                border: none;
                border-radius: 5px;
                padding: 10px;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #ff4d26; /* Promijenite boju ovdje */
            }
        '''

        self.button = QPushButton("Unesi datoteku za obrađivanje", self)
        self.button.setStyleSheet(button_styles)
        self.button.setGeometry(60, 50, 270, 40)
        self.button.clicked.connect(self.import_files)
        
       





    def import_files(self):
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        file_dialog = QFileDialog()
        file_dialog.setOptions(options)
        file_dialog.setFileMode(QFileDialog.ExistingFile)
        file_dialog.setNameFilter("CSV Files (*.csv)")

        if file_dialog.exec_():
            filenames = file_dialog.selectedFiles()
            self.process_files(filenames[0])

    def process_files(self, filename):
        self.button.hide()
        global df, new_df
        df = pd.read_csv(filename, encoding='cp1250', delimiter=";")

        if not df.empty:
            
            mjesta = df['Mjesto'].unique()

           
            df['Mjesto'] = df['Mjesto'].replace({mjesto: mjesta[i] for i, mjesto in enumerate(mjesta)})
            df['Datum'] = pd.date_range(start=datetime(2023, 4, 1), periods=len(df))
            df['Trenutnovrijeme'] = df['Trenutnovrijeme'].replace({
                'Kiša': 0,
                'Oblačno': 1,
                'Poluoblačno': 2,
                'Slabi pljuskovi uz kišu': 3,
                'Sunčano': 4,
                'Uglavnom oblačno': 5,
                'Uglavnom sunčano': 6,
                'Vedro': 7,
                'Vedro s povremenom naoblakom': 8
            })
            X = ['TrenutačnaTemperatura', 'Oborine', 'Vlaga', 'Vjetar', 'Trenutnovrijeme', 'Najvećatemperatura',
                 'Najmanjatemperatura']
            y = ['TrenutačnaTemperatura', 'Oborine', 'Vlaga', 'Vjetar', 'Trenutnovrijeme', 'Najvećatemperatura',
                 'Najmanjatemperatura']

       
            X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.3, random_state=43)

            
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

           
            mlp = MLPRegressor(hidden_layer_sizes=(100, 50, 100, 50), max_iter=1500, solver='adam', activation='relu',
                               random_state=43)
            mlp.fit(X_train, y_train)

            
            y_pred = mlp.predict(X_test)

            
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            

  
            output = f"Srednja kvadratna greška: {mse}\n"
            output += f"R^2: {r2}\n\n"
            

            
            new_dates = pd.date_range(start=datetime(2024, 4, 1), periods=len(df))
            new_df = df.copy()

            new_X = scaler.transform(new_df[X])
            new_y = mlp.predict(new_X)

            for i, param in enumerate(y):
                new_df[param] = new_y[:, i]

           
            trenutnovrijeme_dict = {
                0: 'Kiša',
                1: 'Oblačno',
                2: 'Poluoblačno',
                3: 'Slabi pljuskovi uz kišu',
                4: 'Sunčano',
                5: 'Uglavnom oblačno',
                6: 'Uglavnom sunčano',
                7: 'Vedro',
                8: 'Vedro s povremenom naoblakom'
            }

            merged_df = pd.concat([df, new_df], ignore_index=True).round()
            merged_df['Trenutnovrijeme'] = merged_df['Trenutnovrijeme'].replace(trenutnovrijeme_dict)
            merged_df['Oborine'] = merged_df['Oborine'].apply(lambda x: abs(x))
            new_df_2024 = merged_df.iloc[len(df):].reset_index(drop=True)
            new_dates_2024 = pd.date_range(start=pd.Timestamp(2024, 4, 1), periods=len(new_df_2024), freq='D')
            new_df_2024['Datum'] = new_dates_2024
            new_df_2024 = new_df_2024[y].round(2)
            new_df_2024.insert(7, 'Datum', new_dates_2024)
            new_df_2024['Mjesto'] = df['Mjesto']
            new_df_2024['Dan'] = df['Dan']
            
            table_html = new_df_2024.to_html(index=False)

       
            output += f"Predikcija za 2024 sa svim parametrima:\n\n\n"
            output += table_html

       
            self.text_edit.setGeometry(25, 50, 1500, 1000)  
            self.text_edit.setHtml(output)
            self.text_edit.setStyleSheet("font-size: 12pt;")

           

        
        

if __name__ == "__main__":
    app = QApplication([])
    window = MainWindow()
    window.show()
    app.exec_()
    




Mean Squared Error: 1.550027301975588
R^2: 0.9772918039864524
